In [1]:
import yfinance as yf
import pandas as pd 
import sklearn.model_selection as sk
import numpy as np
from ignite.engine import Engine, Events
from ignite.handlers import EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
import plotly.express as px
data = yf.download('HEKTS.IS',period="1y",interval="1D")
data=data.rename({"High":"high","Low":"low","Close":"close","Open":"open","Adj Close":"adj close","Volume":"volume"},axis=1)

data

# Convert adjusted closing price to numpy array



[*********************100%%**********************]  1 of 1 completed


,open,high,low,close,adj close,volume
Date,,,,,,
2022-10-20,30.600000,31.100000,29.180000,30.020000,30.020000,75573981
2022-10-21,30.020000,30.139999,28.740000,29.500000,29.500000,39818081
2022-10-24,29.600000,29.860001,28.219999,28.959999,28.959999,36113748
2022-10-25,28.959999,29.180000,26.639999,28.000000,28.000000,48810621
2022-10-26,28.040001,29.639999,27.680000,28.940001,28.940001,42724267
...,...,...,...,...,...,...
2023-10-16,24.500000,24.760000,23.120001,23.160000,23.160000,43659210
2023-10-17,23.240000,23.760000,22.900000,23.760000,23.760000,38244470
2023-10-18,23.240000,23.600000,22.719999,22.719999,22.719999,36201053


In [2]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Candlestick(x=data.index,open=data.open,close=data.close,high=data.high,low=data.low)
)
fig.add_trace(go.Bar(x=data.index, y=data['volume'], name='Volume'),secondary_y=True)
fig.update_yaxes(range=[0,1500000000],secondary_y=True)
fig.update_yaxes(visible=False, secondary_y=True)


In [3]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint

def find_grad_intercept(case, x, y):
    
    pos = np.argmax(y) if case == 'resistance' else np.argmin(y)
        
    # Form the points for the objective function
    X = x-x[pos]
    Y = y-y[pos]
    
    if case == 'resistance':
        const = LinearConstraint(
            X.reshape(-1, 1),
            Y,
            np.full(X.shape, np.inf),
        )
    else:
        const = LinearConstraint(
            X.reshape(-1, 1),
            np.full(X.shape, -np.inf),
            Y,
        )
    
    # Min the objective function with a zero starting point for the gradient
    ans = minimize(
        fun = lambda m: np.sum((m*X-Y)**2),
        x0 = [0],
        jac = lambda m: np.sum(2*X*(m*X-Y)),
        method = 'SLSQP',
        constraints = (const),
    )
    
    # Return the gradient (m) and the intercept (c)
    return ans.x[0], y[pos]-ans.x[0]*x[pos] 

In [6]:
import yfinance as yf
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default='browser'


df = yf.download('TSLA').reset_index()

# Perform the date filtering to get the plotting df, and the df to get the 
# trend line of
df = (
      df[(df['Date'] > '2020-05-01') & (df['Date'] < '2020-10-01')]
      .reset_index(drop = True)
)
trend_line_df = df[(df['Date'] > '2020-07-13') & (df['Date'] < '2020-08-11')]

# Using the trend-line algorithm, deduce the gradient and intercept terms of
# the straight lines
m_res, c_res = find_grad_intercept(
    'resistance', 
    trend_line_df.index.values, 
    trend_line_df.High.values,
)
m_supp, c_supp = find_grad_intercept(
    'support', 
    trend_line_df.index.values, 
    trend_line_df.Low.values,
)

# Plot the figure with plotly
layout = go.Layout(
    title = 'TSLA Stock Price',
    xaxis = {'title': 'Date'},
    yaxis = {'title': 'Price'},
) 

fig = go.Figure(
    layout=layout,
    data=[
        go.Candlestick(
            x = df['Date'],
            open = df['Open'], 
            high = df['High'],
            low = df['Low'],
            close = df['Close'],
            name = 'Candlestick chart'
        ),
        go.Scatter(
            x = trend_line_df['Date'],
            y = m_res*trend_line_df.index + c_res,
            name = 'Resistance line'
        ),
        go.Scatter(
            x = trend_line_df['Date'],
            y = m_supp*trend_line_df.index + c_supp,
            name = 'Support line'
        ),
    ]
)


fig.update_xaxes(
        rangeslider_visible = False,
        rangebreaks = [{'bounds': ['sat', 'mon']}]
    )


[*********************100%%**********************]  1 of 1 completed
